In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

In [2]:
df = pd.read_csv("data/data.csv")
df.pop("Diabetes_binary")

columns = df.columns.values
rand = random.randint(11, len(columns))

features = df.values[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'HighBP': 0.0,
 'HighChol': 1.0,
 'CholCheck': 1.0,
 'BMI': 29.0,
 'Smoker': 1.0,
 'Stroke': 0.0,
 'HeartDiseaseorAttack': 0.0,
 'PhysActivity': 1.0,
 'Fruits': 1.0,
 'Veggies': 0.0,
 'HvyAlcoholConsump': 0.0,
 'AnyHealthcare': 1.0,
 'NoDocbcCost': 0.0,
 'GenHlth': 1.0,
 'MentHlth': 0.0,
 'PhysHlth': 0.0,
 'DiffWalk': 1.0,
 'Sex': 0.0,
 'Age': 10.0,
 'Education': 5.0,
 'Income': 1.0}

In [3]:
def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )

In [4]:
def prepare_json(inputs: dict):
    feature_spec = dict()
    
    for keys, values in inputs.items():
        feature_spec[keys] = float_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [5]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)
    
    endpoint = "https://diabetes-pipeline.up.railway.app/v1/models/diabetes-pipeline:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    
    if prediction < 0.6:
        return "No Diabetes"
    else:
        return "Diabetes"

In [6]:
make_predictions(inputs)

'No Diabetes'